## Logits of second model and classification results

Assuming confidence of the first model is one at all instances, Which Is exactly what I did, then we do not even need the first stage to calculate the second stage. We can just use the logits directly. 

Wrong!!!! YOu did not get the logits output correctly. 

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')
sys.path.append('../resnet_model')

import pickle
import os
import numpy as np
import json
import torch

from utils.general.dataset_variables import TripletSegmentationVariables 
from utils.general.read_files import read_from_json
from utils.general.save_files import save_to_json

INSTRUMENT_ID_TO_CLASS_DICT = TripletSegmentationVariables.categories['instrument']
INSTRUMENT_CLASS_TO_ID_DICT = {instrument_class: instrument_id for instrument_id, instrument_class in INSTRUMENT_ID_TO_CLASS_DICT.items()}
TRIPLET_ID_TO_CLASS_DICT = TripletSegmentationVariables.categories['triplet']
TRIPLET_NAME_TO_ID_DICT = {triplet_class: triplet_id for triplet_id, triplet_class in TRIPLET_ID_TO_CLASS_DICT.items()}
VERBTARGET_DICT = TripletSegmentationVariables.categories['verbtarget'] 

INSTRUMENT_TO_VERBTARGET_CLASSES = TripletSegmentationVariables.instrument_to_verbtarget_classes
INSTRUMENT_TO_TRIPLET_CLASSES = TripletSegmentationVariables.instrument_to_triplet_classes


# Softmax needed

In [2]:
def softmax(x):
    x = np.array(x)
    e_x = np.exp(x - np.max(x))  # for numerical stability
    return e_x / e_x.sum()

## My verbtargets and verb and targets - I use softmax

In [3]:
def load_valid_triplets(path="maps.txt"):
    triplets = []
    with open(path, 'r') as f:
        for line in f:
            if line.strip().startswith('# IVT'):
                continue  # skip header
            ivt, i, v, t, _, _ = line.strip().split(',')
            triplets.append((int(ivt), int(i), int(v), int(t)))
    return triplets

In [4]:
len(load_valid_triplets(path="../utils/general/maps.txt"))

100

In [ ]:
# # multitask instrument verb and target
# def generate_softmax_score_for_detected_objects_from_multitask_instrument_verb_and_target_logit(logits_instrument, logits_verb, logits_target, tool_id, triplet_map):   
    
#     instrument_logits = np.array(logits_instrument) 
#     verb_logits = np.array(logits_verb)
#     target_logits =  np.array(logits_target)
    
#     instrument_probs = softmax(verb_logits)
#     verb_probs = softmax(verb_logits)
#     target_probs = softmax(target_logits)
    
   
#     triplet_softmax = np.zeros(len(triplet_map))
    
#     for idx, (ivt, i, v, t) in enumerate(triplet_map):
#         if i == tool_id:
#             triplet_softmax[ivt] = instrument_probs[i] * verb_probs[v] * target_probs[t]
    
#     triplet_softmax = triplet_softmax / triplet_softmax.sum()      

#     predicted_triplet_index = int(np.argmax(triplet_softmax)) + 1  # if triplet_dict is 1-based
#     predicted_triplet_name = TRIPLET_ID_TO_CLASS_DICT[str(predicted_triplet_index)]

#     return triplet_softmax, predicted_triplet_name

In [6]:
# multitask verb and target
def generate_softmax_score_for_detected_objects_from_multitask_verb_and_target_logit(logits_verb, logits_target, tool_id, triplet_map):   
    verb_logits = np.array(logits_verb)
    target_logits =  np.array(logits_target)

    verb_probs = softmax(verb_logits)
    target_probs = softmax(target_logits)
    
    triplet_softmax = np.zeros(len(triplet_map))
    
    for idx, (ivt, i, v, t) in enumerate(triplet_map):
        if i == tool_id:
            triplet_softmax[ivt] = verb_probs[v] * target_probs[t]
    
    triplet_softmax = triplet_softmax / triplet_softmax.sum()      

    predicted_triplet_index = int(np.argmax(triplet_softmax)) + 1  # if triplet_dict is 1-based
    predicted_triplet_name = TRIPLET_ID_TO_CLASS_DICT[str(predicted_triplet_index)]

    return triplet_softmax, predicted_triplet_name

In [7]:
# single task verb_target
def generate_softmax_score_for_detected_objects_from_verbtarget_logit(logits_for_instance, tool_id):
    amount = len(INSTRUMENT_TO_VERBTARGET_CLASSES[tool_id])
    logits = logits_for_instance[:amount]

    assert not np.isinf(logits).any()

    triplet_indices = INSTRUMENT_TO_TRIPLET_CLASSES[tool_id]
    assert len(logits) == len(triplet_indices)

    triplet_logits = np.full(100, -np.inf)
    for idx, logit in zip(triplet_indices, logits):
        triplet_logits[idx] = logit

    triplet_softmax = softmax(triplet_logits)
    predicted_triplet_index = int(np.argmax(triplet_softmax)) + 1  # if triplet_dict is 1-based
    predicted_triplet_name = TRIPLET_ID_TO_CLASS_DICT[str(predicted_triplet_index)]

    return triplet_softmax, predicted_triplet_name


In [8]:
def get_ivt_scores_from_my_logits(logits_file_path, task='verbtarget', triplet_map_path="../utils/general/maps.txt"):
    '''
    options would be 
    
    '''
    second_stage_logits = read_from_json(logits_file_path)

    results = {}

    for instance_key, logit_dict in second_stage_logits.items():
        image_name, tool_name, *_ = instance_key.split(',')
        image_name = image_name.replace('t50_', '')
        tool_id = int(INSTRUMENT_CLASS_TO_ID_DICT[tool_name]) - 1

        if task == 'verbtarget':
            logits = logit_dict['logits_verbtarget']
            softmax_vector, predicted_triplet_name = generate_softmax_score_for_detected_objects_from_verbtarget_logit(logits, tool_id)
        elif task ==  'verb_and_target_multitask':  
            logits_verb = logit_dict['logits_verb']
            logits_target = logit_dict['logits_target']
            triplet_map = load_valid_triplets(path=triplet_map_path)
            softmax_vector, predicted_triplet_name = generate_softmax_score_for_detected_objects_from_multitask_verb_and_target_logit(logits_verb, logits_target, tool_id, triplet_map)
        elif task ==  'instrument_verb_and_target_multitask': 
            logits_instrument = logit_dict['logits_instrument']
            logits_verb = logit_dict['logits_verb']
            logits_target = logit_dict['logits_target']
            triplet_map = load_valid_triplets(path=triplet_map_path)
            softmax_vector, predicted_triplet_name = generate_softmax_score_for_detected_objects_from_multitask_instrument_verb_and_target_logit(logits_instrument, logits_verb, logits_target, tool_id, triplet_map)   
             
              

        if image_name not in results:
            results[image_name] = {
                'triplet_prediction': softmax_vector,
                'triplet_name': [predicted_triplet_name]  # list of per-instance predictions
            }
        else:
            results[image_name]['triplet_prediction'] = np.maximum(
                results[image_name]['triplet_prediction'],
                softmax_vector
            )
            results[image_name]['triplet_name'].append(predicted_triplet_name)

    # Convert prediction vector to list for JSON
    for image_name in results:
        results[image_name]['triplet_prediction'] = results[image_name]['triplet_prediction'].tolist()

    return results


In [15]:
# work_dir = '../resnet_model/results_from_dgx/v3/multitask_resnet_fpn' 
# second_stage_logits_file_path = f'{work_dir}/results_logits.json'
# save_path_for_softmax_scores = f'{work_dir}/results_triplet_softmax_scores.json'


# work_dir = '../resnet_model/results_from_dgx/v3/multitask_resnet_fpn_transformer_decoder' 
# second_stage_logits_file_path = f'{work_dir}/results_logits.json'
# save_path_for_softmax_scores = f'{work_dir}/results_triplet_softmax_scores.json'


work_dir = '../resnet_model/results_from_dgx/v3/singletask_resnet_fpn_verbtarget' 
second_stage_logits_file_path = f'{work_dir}/results_logits.json'
save_path_for_softmax_scores = f'{work_dir}/results_triplet_softmax_scores.json'



In [16]:
triplet_sigmoid_for_all_results = get_ivt_scores_from_my_logits(logits_file_path = second_stage_logits_file_path,
                                                                task = 'verbtarget' )

save_to_json(data=triplet_sigmoid_for_all_results,
             json_file_path=save_path_for_softmax_scores)

## Four task prediction

In [21]:
def generate_softmax_score_for_detected_objects_from_logit_ivt(triplet_logits):
    
    triplet_softmax = softmax(triplet_logits)
    predicted_triplet_index = int(np.argmax(triplet_softmax)) + 1  # if triplet_dict is 1-based
    predicted_triplet_name = TRIPLET_ID_TO_CLASS_DICT[str(predicted_triplet_index)]

    return triplet_softmax, predicted_triplet_name


In [24]:
def get_scores_from_my_second_stage_logits_ivt(second_stage_logits_file_path):
    second_stage_logits = read_from_json(second_stage_logits_file_path)

    results = {}

    for instance_key, logit_dict in second_stage_logits.items():
        image_name, instance_name, = instance_key.split('_instance_')
        image_name = image_name.replace('t50_', '')

        triplet_logits = logit_dict['logits_ivt']
        softmax_vector, predicted_triplet_name = generate_softmax_score_for_detected_objects_from_logit_ivt(triplet_logits)

        if image_name not in results:
            results[image_name] = {
                'triplet_prediction': softmax_vector,
                'triplet_name': [predicted_triplet_name]  # list of per-instance predictions
            }
        else:
            results[image_name]['triplet_prediction'] = np.maximum(
                results[image_name]['triplet_prediction'],
                softmax_vector
            )
            results[image_name]['triplet_name'].append(predicted_triplet_name)

    # Convert prediction vector to list for JSON
    for image_name in results:
        results[image_name]['triplet_prediction'] = results[image_name]['triplet_prediction'].tolist()

    return results


In [25]:
second_stage_logits_file_path = '../resnet_model/work_dirs/fourtask_moe/results_logits.json'
triplet_sigmoid_for_all_results = get_scores_from_my_second_stage_logits_ivt(second_stage_logits_file_path)

save_path_for_sigmoid_scores = '../resnet_model/work_dirs/fourtask_moe/results_triplet_softmax_scores.json'
save_to_json(data=triplet_sigmoid_for_all_results,
             json_file_path=save_path_for_sigmoid_scores)

## For Triplet from MMdet

In [3]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')
sys.path.append('../resnet_model')

import pickle
import os
import numpy as np
import json

from utils.general.dataset_variables import TripletSegmentationVariables 
from utils.general.read_files import read_from_json
from utils.general.save_files import save_to_json



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
def get_scores_from_my_mmdet_logits(mmdet_logits_file_path):
    mmdet_logits = read_from_json(mmdet_logits_file_path)

    results = {}

    for image_name, logit_dict_for_all_instances in mmdet_logits.items():
        image_name = image_name.replace('t50_', '')
        
        if logit_dict_for_all_instances is None:
            continue
        
        for  logit_dict_per_instance in logit_dict_for_all_instances:
            softmax_vector = logit_dict_per_instance['softmax']
            predicted_triplet_name = logit_dict_per_instance['predicted_class_name']
            
            if image_name not in results:
                results[image_name] = {
                    'triplet_prediction': np.array(softmax_vector),
                    'triplet_name': [predicted_triplet_name] 
                }
            else:
                results[image_name]['triplet_prediction'] = np.maximum(
                    results[image_name]['triplet_prediction'],
                    softmax_vector
                )
                results[image_name]['triplet_name'].append(predicted_triplet_name)

    # Convert prediction vector to list for JSON
    for image_name in results:
        results[image_name]['triplet_prediction'] = results[image_name]['triplet_prediction'].tolist()

    return results


In [7]:
# mmdet_logits_file_path = '../results/mask2former_direct_triplet_prediction/predicted_logits.json'
# mmdet_logits_file_path = '../results/triplet_direct_v2/predicted_logits.json'
# save_path_for_scores = '../results/triplet_direct_v2/results_triplet_softmax_scores.json'

mmdet_logits_file_path = '../results/multiple_heads_v2/predicted_logits.json'
save_path_for_scores = '../results/multiple_heads_v2/results_triplet_softmax_scores.json'
triplet_sigmoid_for_all_results = get_scores_from_my_mmdet_logits(mmdet_logits_file_path)


save_to_json(data=triplet_sigmoid_for_all_results,
             json_file_path=save_path_for_scores)